In [10]:
import polars as pl

quarterly_targets_file = 'https://drive.google.com/u/0/uc?id=1YNg4R6Ud_Dxi108VJvxndlQNe5Rn9IdI&export=download'
transactions_file = 'https://drive.google.com/u/0/uc?id=10EYKJ989gReQJhOGHx9vK7iPOHii-g_q&export=download'
pl.Config.set_fmt_str_lengths(50)

targets_df=pl.read_csv(quarterly_targets_file)
transactions_df=pl.read_csv(transactions_file, try_parse_dates=True)


In [11]:
targets_df.head()

Online or In-Person,Q1,Q2,Q3,Q4
str,i64,i64,i64,i64
"""Online""",72500,70000,60000,60000
"""In-Person""",75000,70000,70000,60000


In [12]:
transactions_df.head(1)

Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
str,i64,i64,i64,datetime[μs]
"""DTB-716-679-576""",1448,100001,2,2023-03-20 00:00:00


In [13]:
transactions_df.with_columns(
    pl.col('Transaction Code').str.n_chars().max().alias('max_chars'),
    pl.col('Transaction Code').str.n_chars().min().alias('min_chars')
)

Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date,max_chars,min_chars
str,i64,i64,i64,datetime[μs],u32,u32
"""DTB-716-679-576""",1448,100001,2,2023-03-20 00:00:00,15,14
"""DS-795-814-303""",7839,100001,2,2023-11-15 00:00:00,15,14
"""DSB-807-592-406""",5520,100005,1,2023-07-14 00:00:00,15,14
"""DS-367-545-264""",7957,100007,2,2023-08-18 00:00:00,15,14
"""DSB-474-374-857""",5375,100000,2,2023-08-26 00:00:00,15,14
"""DSB-448-546-348""",4525,100009,1,2023-05-27 00:00:00,15,14
"""DS-213-969-866""",2321,100007,2,2023-01-04 00:00:00,15,14
"""DS-341-615-729""",1967,100004,1,2023-01-22 00:00:00,15,14
"""DTB-200-951-620""",8105,100003,2,2023-01-10 00:00:00,15,14


In [14]:
targets_df.describe()

describe,Online or In-Person,Q1,Q2,Q3,Q4
str,str,f64,f64,f64,f64
"""count""","""2""",2.0,2.0,2.0,2.0
"""null_count""","""0""",0.0,0.0,0.0,0.0
"""mean""",null,73750.0,70000.0,65000.0,60000.0
"""std""",null,1767.766953,0.0,7071.067812,0.0
"""min""","""In-Person""",72500.0,70000.0,60000.0,60000.0
"""max""","""Online""",75000.0,70000.0,70000.0,60000.0
"""median""",null,73750.0,70000.0,65000.0,60000.0
"""25%""",null,72500.0,70000.0,60000.0,60000.0
"""75%""",null,75000.0,70000.0,70000.0,60000.0


In [17]:
transactions_df=transactions_df.with_columns(
    pl.col('Transaction Code').str.splitn('-',2),
    pl.col('Online or In-Person').map_dict({1:'Online',2:'In-Person'})
).unnest('Transaction Code')

transactions_df=transactions_df.filter(
    pl.col('field_0') == 'DSB'
)

In [18]:
# transactions_df.with_columns(
#     pl.col('Online or In-Person').map_dict({1:'Online',2:'In-Person'})
# )

transactions_df.head(1)

field_0,field_1,Value,Customer Code,Online or In-Person,Transaction Date
str,str,i64,i64,str,datetime[μs]
"""DSB""","""807-592-406""",5520,100005,"""Online""",2023-07-14 00:00:00


In [ ]:
transactions_df.with_columns(
    pl.col('Transaction Date').str.strptime(pl.Utf8, format=)
)